In [15]:
import sys
import os
sys.path.insert(0, os.path.abspath('./src/'))

$$         piTM = \max_{i \in \mathcal{I}} \frac{1}{I} \sum_{j \in \mathcal{I}}  \frac{1}{1 + [\langle e_{ij} \rangle / d_0 (I)]^2}
$$ d_0(I) = \begin{cases} \mbox{true,} & \mbox{if } 

$$ d_0(I) = \begin{cases} 1.25 \sqrt[3]{I} -15 -1.8\text{,} & \text{if } I \geq 22 \\ 0.02 I \text{,} & \text{if } I < 22  \end{cases}

In [17]:
if not os.path.exists('./tmp'):
  os.mkdir('./tmp')

In [18]:
file_in = '../BeEM/example_input/3j6b.cif'

## PDB Numpy

In [19]:
from pdb_numpy import Coor, Model, format, abinitio

test = Coor(file_in)
test.write("tmp.pdb", overwrite=True)

In [20]:
%timeit Coor(file_in)

2.51 s ± 15.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%%timeit

test = Coor(file_in)
test.write("tmp.pdb", overwrite=True)

5.19 s ± 56.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## BeEM

In [23]:
import subprocess

In [25]:
os.chdir('tmp')
%timeit subprocess.run(["../../BeEM/BeEM", "../" + file_in,])
os.chdir('..')

3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt
3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt
3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt
3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt
3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt
3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt
3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt
3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt
1.16 s ± 15.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Biopython

In [12]:
# First import MMCIF2Dict function

from Bio.PDB import PDBIO, MMCIFParser

# read in the CIF file.
parser = MMCIFParser()
struc = parser.get_structure('3j6b', filename=file_in)

# Save as pdb
io=PDBIO()
io.set_structure(struc)
io.save("tmp_biopython.pdb")

/home/murail/miniforge3/envs/docking/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 195024.
  warnings.warn(
/home/murail/miniforge3/envs/docking/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 195133.
  warnings.warn(
/home/murail/miniforge3/envs/docking/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain N is discontinuous at line 195134.
  warnings.warn(
/home/murail/miniforge3/envs/docking/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain W is discontinuous at line 195135.
  warnings.warn(
/home/murail/miniforge3/envs/docking/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain 0 is discontinuous at line 195136.
  warnings.warn(


PDBIOException: Error when writing atom ('3j6b', 0, 'E', (' ', 83, ' '), ('CD2', ' '))

In [18]:

class OutOfChainsError(Exception): pass
def rename_chains(structure):
    """Renames chains to be one-letter chains
    
    Existing one-letter chains will be kept. Multi-letter chains will be truncated
    or renamed to the next available letter of the alphabet.
    
    If more than 62 chains are present in the structure, raises an OutOfChainsError
    
    Returns a map between new and old chain IDs, as well as modifying the input structure
    """
    next_chain = 0 #
    # single-letters stay the same
    chainmap = {c.id:c.id for c in structure.get_chains() if len(c.id) == 1}
    for o in structure.get_chains():
        if len(o.id) != 1:
            if o.id[0] not in chainmap:
                chainmap[o.id[0]] = o.id
                o.id = o.id[0]
            else:
                c = int_to_chain(next_chain)
                while c in chainmap:
                    next_chain += 1
                    c = int_to_chain(next_chain)
                    if next_chain >= 62:
                        raise OutOfChainsError()
                chainmap[c] = o.id
                o.id = c
    return chainmap

In [1]:
from Bio.PDB import MMCIFParser, PDBIO

parser = MMCIFParser()
struc = parser.get_structure('3j6b', filename=file_in)

chainmap = rename_chains(struc)

io = PDBIO()
io.set_structure(struc)
io.save("tmp_biopython.pdb")

NameError: name 'file_in' is not defined

In [20]:
%timeit parser.get_structure('3j6b', filename=file_in)

/home/murail/miniforge3/envs/docking/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 195024.
  warnings.warn(
/home/murail/miniforge3/envs/docking/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 195133.
  warnings.warn(
/home/murail/miniforge3/envs/docking/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain N is discontinuous at line 195134.
  warnings.warn(
/home/murail/miniforge3/envs/docking/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain W is discontinuous at line 195135.
  warnings.warn(
/home/murail/miniforge3/envs/docking/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain 0 is discontinuous at line 195136.
  warnings.warn(
/home/murail/miniforge3/envs/docking/lib/python3.10/sit

7.24 s ± 62.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit

test = Coor(file_in)
test.write("tmp.pdb", overwrite=True)